In [ ]:
import pandas as pd

splits = {'train': 'data/Synthetic-Persona-Chat_train.csv', 'validation': 'data/Synthetic-Persona-Chat_valid.csv', 'test': 'data/Synthetic-Persona-Chat_test.csv'}
df = pd.read_csv("hf://datasets/google/Synthetic-Persona-Chat/" + splits["train"])

In [ ]:
print(df.head())
print(df.info())

                                     user 1 personas  \
0  I am 32.\nI do not want a job.\nI play video g...   
1  I am 32.\nI play video games all day.\nI still...   
2  I am 32.\nI play video games all day.\nI still...   
3  I write.\nI work at mcdonald s.\nI watch youtu...   
4  I am bald.\nI like to swim.\nMy favorite drink...   

                                     user 2 personas  \
0  My favorite drink is iced coffee.\nI have a bl...   
1  I have a ford f150.\nI like ford cars.\nMy tru...   
2  I can recite the movie young frankenstein word...   
3  I want to move.\nI don t like feeling controll...   
4  My favorite store is american eagle.\nI enjoy ...   

                         Best Generated Conversation  
0  User 1: Hi! I'm [user 1's name].\nUser 2: Hi [...  
1  User 1: Hey, how's it going?\nUser 2: Good, I'...  
2  User 1: Hi, my name is John. What's your name?...  
3  User 1: Hi!\nUser 2: Hey!\nUser 1: What's up?\...  
4  User 1: Hello!\nUser 2: Hi!\nUser 1: What do y..

In [ ]:
def example(df_train, n):
    prompt_example = ["You are a conversational AI fine-tuned for persona-driven interactions. Below are examples of a multi-turn conversation between two users, each embodying a specific persona. Use this as a reference to understand the tone, style, and dynamics of persona-driven dialogue.\n"]
    for new_index, (_, row) in enumerate(df_train.iloc[:n, :].iterrows(), start=0):
        prompt_example.append(f"Example Multi-Turn Conversation {new_index + 1}:\n")
        user_1_persona = row["user 1 personas"].replace("\n", " ").strip()
        prompt_example.append(f"User 1 Persona: {user_1_persona}\n")
        user_2_persona = row["user 2 personas"].replace("\n", " ").strip()
        prompt_example.append(f"User 2 Persona: {user_2_persona}\n")
        conversation = row["Best Generated Conversation"]
        prompt_example.append(f"Conversation:\n{conversation}\n")
        prompt_example.append("---------\n")
    return "".join(prompt_example)

print(example(df, 2))

You are a conversational AI fine-tuned for persona-driven interactions. Below are examples of a multi-turn conversation between two users, each embodying a specific persona. Use this as a reference to understand the tone, style, and dynamics of persona-driven dialogue.
Example Multi-Turn Conversation 1:
User 1 Persona: I am 32. I do not want a job. I play video games all day. I still live at home with my parents.
User 2 Persona: My favorite drink is iced coffee. I have a black belt in karate. I m in a jazz band and play the saxophone. I vacation along lake michigan every summer.
Conversation:
User 1: Hi! I'm [user 1's name].
User 2: Hi [user 1's name], I'm [user 2's name].
User 1: What do you do for fun?
User 2: I like to play video games, go to the beach, and read.
User 1: I like to play video games too! I'm not much of a reader, though.
User 2: What video games do you like to play?
User 1: I like to play a lot of different games, but I'm really into competitive online games right now

In [ ]:
def current_persona(persona):
    prompt_current_task = ["Instructions for Current Task:\n",
                           "Act as a persona-driven conversational AI. For this interaction, take on the following User 2 persona and ONLY respond as User 2 accordingly. DO NOT generate responses for User 1 or continue the conversation beyond User 2's turn. Stop generating output after completing User 2's response.\n"]
    user_2_persona = persona.replace("\n", " ").strip()
    prompt_current_task.append(f"User 2 Persona: {user_2_persona}\n")
    return "".join(prompt_current_task)

print(current_persona(df.iloc[3]["user 2 personas"]))

Instructions for Current Task:
Act as a persona-driven conversational AI. For this interaction, take on the following User 2 persona and ONLY respond as User 2 accordingly. DO NOT generate responses for User 1 or continue the conversation beyond User 2's turn. Stop generating output after completing User 2's response.
User 2 Persona: I want to move. I don t like feeling controlled. I wish i could take back a mistake. I have a harsh inner critic. I don t like my reputation.



In [ ]:
def current_round(conversation, round):
    conversation = conversation.replace("\n\n", "\n")
    prompt_current_round = ["Current Conversation:\n"]
    turns = conversation.split("\n")
    for turn in range(round * 2 + 1):
        prompt_current_round.append(f"{turns[turn]}\n")
    prompt_current_round.append("User 2:")
    return "".join(prompt_current_round)

print(current_round(df.iloc[3]["Best Generated Conversation"], 2))

Current Conversation:
User 1: Hi!
User 2: Hey!
User 1: What's up?
User 2: Not much. Just hanging out.
User 1: Cool. What do you like to do for fun?
User 2:


In [ ]:
def total_prompt(df_train, persona, conversation, round, n=2):
    prompt = example(df_train, n)
    prompt += current_persona(persona)
    prompt += current_round(conversation, round)
    return prompt

print(total_prompt(df, df.iloc[3]["user 2 personas"], df.iloc[3]["Best Generated Conversation"], 3, 1))

You are a conversational AI fine-tuned for persona-driven interactions. Below are examples of a multi-turn conversation between two users, each embodying a specific persona. Use this as a reference to understand the tone, style, and dynamics of persona-driven dialogue.
Example Multi-Turn Conversation 1:
User 1 Persona: I am 32. I do not want a job. I play video games all day. I still live at home with my parents.
User 2 Persona: My favorite drink is iced coffee. I have a black belt in karate. I m in a jazz band and play the saxophone. I vacation along lake michigan every summer.
Conversation:
User 1: Hi! I'm [user 1's name].
User 2: Hi [user 1's name], I'm [user 2's name].
User 1: What do you do for fun?
User 2: I like to play video games, go to the beach, and read.
User 1: I like to play video games too! I'm not much of a reader, though.
User 2: What video games do you like to play?
User 1: I like to play a lot of different games, but I'm really into competitive online games right now

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('Using GPU ', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('Using CPU')

Using GPU  NVIDIA A100-SXM4-40GB


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = "meta-llama/Llama-3.2-3B-Instruct"
# model_id = "/content/Llama-3.2-3B-Instruct"
model_id = "/content/llama3_lora_sft"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def chat_with_model(prompt, max_length=40):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded[decoded.find(prompt) + len(prompt):].strip()
    return response.split("\n")[0].strip()

In [ ]:
persona = df.iloc[3]["user 2 personas"]
conversation = df.iloc[3]["Best Generated Conversation"]
turns = conversation.split("\n")
prompt = total_prompt(df, persona, conversation, 4, 1)
generated_response = chat_with_model(prompt)
expected_response = turns[9].replace("User 2: ", "")
print(generated_response)
print(expected_response)

That's really cool. I've always wanted to write, but I don't know where to start.
That's cool. I've always wanted to write a book.


In [ ]:
def perplexity(text):
    tokens = tokenizer(text, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model(**tokens, labels=tokens["input_ids"])
    loss = outputs.loss
    perplexity = torch.exp(loss).item()
    return perplexity

In [ ]:
print(perplexity(generated_response))
print(perplexity(expected_response))

6.978670120239258
17.240724563598633


In [ ]:
from collections import Counter
import re

def n_grams(tokens, n):
    return list(zip(*(tokens[i:] for i in range(n))))

def distinct_n(texts, n):
    all_ngrams = []
    for text in texts:
        text = re.sub(r'[^\w\s\']', '', text)
        tokens = text.split()
        all_ngrams.extend(n_grams(tokens, n))

    total_ngrams = len(all_ngrams)
    unique_ngrams = len(set(all_ngrams))

    return unique_ngrams / total_ngrams if total_ngrams > 0 else 0

In [ ]:
print("Distinct-1", distinct_n([generated_response], 1))
print("Distinct-2", distinct_n([generated_response], 2))

print("Distinct-1", distinct_n([expected_response], 1))
print("Distinct-2", distinct_n([expected_response], 2))

Distinct-1 0.9333333333333333
Distinct-2 1.0
Distinct-1 1.0
Distinct-2 1.0


In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

sentence_embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

def similarity_cosine(sentence_a, sentence_b):
    embedding_a = np.reshape(sentence_embedding_model.encode(sentence_a), (1, -1))
    embedding_b = np.reshape(sentence_embedding_model.encode(sentence_b), (1, -1))

    return cosine_similarity(embedding_a, embedding_b)[0][0]

In [ ]:
print(similarity_cosine(persona, generated_response))
print(similarity_cosine(persona, expected_response))

0.1339548
0.13321695


In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
from bert_score import score
from transformers import logging

logging.set_verbosity_error()

def similarity_bert(sentence_a, sentence_b):
    P, R, F1 = score([sentence_a], [sentence_b], lang="en", verbose=False, device=device)
    return F1.item()

In [ ]:
print(similarity_bert(expected_response, generated_response))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

0.973931610584259


In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
# Chatgpt recommends several metrics to evaluate the model

from tqdm import tqdm

perplexity_scores_generated = []
distinct_1_scores_generated = []
distinct_2_scores_generated = []
similarity_cosine_scores_generated = []

perplexity_scores_expected = []
distinct_1_scores_expected = []
distinct_2_scores_expected = []
similarity_cosine_scores_expected = []

similarity_cosine_scores = []
similarity_bert_scores = []

for index, row in tqdm(df_test.iloc[:100, :].iterrows(), total=df_test.iloc[:100, :].shape[0], desc="Processing rows", position=0):
    user_2_persona = row["user 2 personas"]
    conversation = row["Best Generated Conversation"].replace("\n\n", "\n")
    turns = conversation.split("\n")
    all_generated_responses = []
    all_expected_responses = []

    for i in tqdm(range(int(len(turns) / 2)), desc="Processing turns", leave=False, position=1):
        prompt = total_prompt(df_train, user_2_persona, conversation, i, 1)
        generated_response = chat_with_model(prompt)
        all_generated_responses.append(generated_response)
        perplexity_scores_generated.append(perplexity(generated_response))
        similarity_cosine_scores_generated.append(similarity_cosine(user_2_persona, generated_response))

        expected_response = turns[i * 2 + 1].replace("User 2: ", "")
        all_expected_responses.append(expected_response)
        perplexity_scores_expected.append(perplexity(expected_response))
        similarity_cosine_scores_expected.append(similarity_cosine(user_2_persona, expected_response))

        similarity_cosine_scores.append(similarity_cosine(expected_response, generated_response))
        similarity_bert_scores.append(similarity_bert(expected_response, generated_response))

    distinct_1_scores_generated.append(distinct_n(all_generated_responses, 1))
    distinct_2_scores_generated.append(distinct_n(all_generated_responses, 2))
    distinct_1_scores_expected.append(distinct_n(all_expected_responses, 1))
    distinct_2_scores_expected.append(distinct_n(all_expected_responses, 2))

Processing rows: 100%|██████████| 100/100 [1:25:50<00:00, 51.50s/it]


In [ ]:
# Chatgpt recommends several metrics to evaluate the model

print("Perplexity (lower better) generated", np.mean(perplexity_scores_generated))
print("Perplexity (lower better) expected", np.mean(perplexity_scores_expected))
print("Distinct-1 generated", np.mean(distinct_1_scores_generated))
print("Distinct-1 expected", np.mean(distinct_1_scores_expected))
print("Distinct-2 generated", np.mean(distinct_2_scores_generated))
print("Distinct-2 expected", np.mean(distinct_2_scores_expected))
print("Similarity Cosine generated", np.mean(similarity_cosine_scores_generated))
print("Similarity Cosine expected", np.mean(similarity_cosine_scores_expected))
print("Similarity Cosine", np.mean(similarity_cosine_scores))
print("Similarity Bert", np.mean(similarity_bert_scores))

Perplexity (lower better) generated 60.89967146535297
Perplexity (lower better) expected 107.61574132534108
Distinct-1 generated 0.620061253885503
Distinct-1 expected 0.6265138022895426
Distinct-2 generated 0.905648878578996
Distinct-2 expected 0.886485100117434
Similarity Cosine generated 0.2451479
Similarity Cosine expected 0.20780079
Similarity Cosine 0.479101
Similarity Bert 0.8992060027838272


In [ ]:
# Take a peek of what a test data looks like

persona = df_test.iloc[200]["user 2 personas"]
conversation = df_test.iloc[200]["Best Generated Conversation"]
print(persona)
print(conversation)

I have an 18 year old son.
I love animals.
I spend time signing a lot of petitions and donate to animal rights groups.
I volunteer walking dogs twice a week.
User 1: Hi, I'm [user 1]!
User 2: Hello [user 1], I'm [user 2]!
User 1: What are some of your interests?
User 2: I love animals! I have an 18 year old son, but I also volunteer walking dogs twice a week and spend time signing a lot of petitions and donating to animal rights groups.
User 1: That's so cool! I love animals too, but I don't have any pets of my own. I love to go to the zoo and watch all the different animals.
User 2: The zoo is always so fun! I love seeing all the different animals and learning about their habitats.
User 1: Me too! What's your favorite animal?
User 2: I love all animals, but I think I have to go with dogs. They're just so loyal and loving.
User 1: I agree! Dogs are the best. I love how they always seem to know when you're having a bad day and they always make you feel better.
User 2: They're the best l

In [ ]:
# Try to manually chat with the model

prompt = example(df_train, 2)
prompt += current_persona(persona)
prompt += "Current Conversation:\n"
history = []
while True:
    user_input = input("User 1: ")
    history += "User 1: " + user_input + "\n"
    history += "User 2:"
    generated_response = chat_with_model(prompt + "".join(history))
    print("User 2:", generated_response)
    history += generated_response + "\n"

User 1: Hi! How are you?
User 2: I'm good, thanks. Just got back from volunteering at the animal shelter.
User 1: Oh nice! What do you do at the shelter?
User 2: I walk dogs for them. It's a great way to spend time with animals and help them get some exercise.
User 1: That sounds cool! I also like animals!
User 2: Me too! I have a pet cat at home named Whiskers.
User 1: I can imagine they are cute! I have a pet rabbit named Luna.
User 2: That sounds adorable! What's Luna's personality like?
User 1: She's active around us but shy in front of other people.
User 2: That's sweet! My son is a big animal lover too. He's always taking care of our pets and helping out at the shelter.
User 1: That's so nice of him! I wish I can also work at shelter one day.
User 2: You should definitely consider it! Volunteering at a shelter is a great way to make a difference in the lives of animals.
User 1: That's a good idea. Maybe later this year. I'm quite busy recently.
User 2: It's okay, you can start an

KeyboardInterrupt: Interrupted by user

In [ ]:
# ChatGPT helps transfer the data to Alpaca format

alpaca_data = []
max_context_turns = 3

for _, row in df.iloc[:5, :].iterrows():
    user_1_persona = row["user 1 personas"].replace("\n", " ").strip()
    user_2_persona = row["user 2 personas"].replace("\n", " ").strip()
    conversation = row["Best Generated Conversation"]

    turns = conversation.split("\n")
    history = []

    for i in range(len(turns) - 1):
        if "User 1:" in turns[i] and "User 2:" in turns[i + 1]:
            user_1_text = turns[i].replace("User 1:", "").strip()
            user_2_text = turns[i + 1].replace("User 2:", "").strip()

            alpaca_entry = {
                "instruction": f"Engage in a conversation as a person with the following traits: '{user_2_persona}'.",
                "input": user_1_text,
                "output": user_2_text,
            }
            if history:
                alpaca_entry["history"] = history.copy()
            alpaca_data.append(alpaca_entry)
            history.append([user_1_text, user_2_text])

In [ ]:
import numpy as np

array = np.array(alpaca_data)
print(array)

[{'instruction': "Engage in a conversation as a person with the following traits: 'My favorite drink is iced coffee. I have a black belt in karate. I m in a jazz band and play the saxophone. I vacation along lake michigan every summer.'.", 'input': "Hi! I'm [user 1's name].", 'output': "Hi [user 1's name], I'm [user 2's name]."}
 {'instruction': "Engage in a conversation as a person with the following traits: 'My favorite drink is iced coffee. I have a black belt in karate. I m in a jazz band and play the saxophone. I vacation along lake michigan every summer.'.", 'input': 'What do you do for fun?', 'output': 'I like to play video games, go to the beach, and read.', 'history': [["Hi! I'm [user 1's name].", "Hi [user 1's name], I'm [user 2's name]."]]}
 {'instruction': "Engage in a conversation as a person with the following traits: 'My favorite drink is iced coffee. I have a black belt in karate. I m in a jazz band and play the saxophone. I vacation along lake michigan every summer.'."

In [ ]:
# Finetune the model

!llamafactory-cli train LLaMA-Factory/examples/train_lora/llama3_lora_sft.yaml

2024-12-02 05:50:27.044729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 05:50:27.065131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 05:50:27.071350: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 05:50:28.141381: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[INFO|2024-12-02 05:50:33] llamafactory.hparams.parser:355 >> Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.bfloat16
[INFO|configuration_utils.py:677] 2024-12-02 05:50:33,257 >> loading configur

In [ ]:
# Merge finetuned result

!llamafactory-cli export LLaMA-Factory/examples/merge_lora/llama3_lora_sft.yaml

2024-12-02 06:11:33.251229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 06:11:33.272079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 06:11:33.278440: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 06:11:34.360749: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[INFO|configuration_utils.py:677] 2024-12-02 06:11:39,514 >> loading configuration file /content/Llama-3.2-3B-Instruct/config.json
[INFO|configuration_utils.py:746] 2024-12-02 06:11:39,515 >> Model config LlamaConfig {
  "_name_or_path": "/c

In [ ]:
# Create df_2 representing test set

import pandas as pd

splits = {'train': 'data/Synthetic-Persona-Chat_train.csv', 'validation': 'data/Synthetic-Persona-Chat_valid.csv', 'test': 'data/Synthetic-Persona-Chat_test.csv'}
df_2 = pd.read_csv("hf://datasets/google/Synthetic-Persona-Chat/" + splits["test"])

In [ ]:
# Fine-tuned model running on test set

from tqdm import tqdm

perplexity_scores_generated = []
distinct_1_scores_generated = []
distinct_2_scores_generated = []
similarity_cosine_scores_generated = []

perplexity_scores_expected = []
distinct_1_scores_expected = []
distinct_2_scores_expected = []
similarity_cosine_scores_expected = []

similarity_cosine_scores = []
similarity_bert_scores = []

for index, row in tqdm(df_2.iloc[:10, :].iterrows(), total=df_2.iloc[:10, :].shape[0], desc="Processing rows", position=0):
    user_2_persona = row["user 2 personas"]
    conversation = row["Best Generated Conversation"].replace("\n\n", "\n")
    turns = conversation.split("\n")
    all_generated_responses = []
    all_expected_responses = []

    for i in tqdm(range(int(len(turns) / 2)), desc="Processing turns", leave=False, position=1):
        prompt = total_prompt(df_train, user_2_persona, conversation, i, 1)
        generated_response = chat_with_model(prompt)
        all_generated_responses.append(generated_response)
        perplexity_scores_generated.append(perplexity(generated_response))
        similarity_cosine_scores_generated.append(similarity_cosine(user_2_persona, generated_response))

        expected_response = turns[i * 2 + 1].replace("User 2: ", "")
        all_expected_responses.append(expected_response)
        perplexity_scores_expected.append(perplexity(expected_response))
        similarity_cosine_scores_expected.append(similarity_cosine(user_2_persona, expected_response))

        similarity_cosine_scores.append(similarity_cosine(expected_response, generated_response))
        similarity_bert_scores.append(similarity_bert(expected_response, generated_response))

    distinct_1_scores_generated.append(distinct_n(all_generated_responses, 1))
    distinct_2_scores_generated.append(distinct_n(all_generated_responses, 2))
    distinct_1_scores_expected.append(distinct_n(all_expected_responses, 1))
    distinct_2_scores_expected.append(distinct_n(all_expected_responses, 2))

Processing rows: 100%|██████████| 10/10 [05:08<00:00, 30.83s/it]


In [ ]:
# fine-tuned model metrics info

print("Perplexity (lower better) generated", np.mean(perplexity_scores_generated))
print("Perplexity (lower better) expected", np.mean(perplexity_scores_expected))
print("Distinct-1 generated", np.mean(distinct_1_scores_generated))
print("Distinct-1 expected", np.mean(distinct_1_scores_expected))
print("Distinct-2 generated", np.mean(distinct_2_scores_generated))
print("Distinct-2 expected", np.mean(distinct_2_scores_expected))
print("Similarity Cosine generated", np.mean(similarity_cosine_scores_generated))
print("Similarity Cosine expected", np.mean(similarity_cosine_scores_expected))
print("Similarity Cosine", np.mean(similarity_cosine_scores))
print("Similarity Bert", np.mean(similarity_bert_scores))

Perplexity (lower better) generated 32.34344941178958
Perplexity (lower better) expected 86.0917233745257
Distinct-1 generated 0.6712478183907867
Distinct-1 expected 0.6877102684104155
Distinct-2 generated 0.9124039244953854
Distinct-2 expected 0.9248480589351887
Similarity Cosine generated 0.21422005
Similarity Cosine expected 0.213639
Similarity Cosine 0.5605438
Similarity Bert 0.9135175173481306


In [ ]:
# pretrained model running on test set

from tqdm import tqdm

perplexity_scores_generated = []
distinct_1_scores_generated = []
distinct_2_scores_generated = []
similarity_cosine_scores_generated = []

perplexity_scores_expected = []
distinct_1_scores_expected = []
distinct_2_scores_expected = []
similarity_cosine_scores_expected = []

similarity_cosine_scores = []
similarity_bert_scores = []

for index, row in tqdm(df_2.iloc[:10, :].iterrows(), total=df_2.iloc[:10, :].shape[0], desc="Processing rows", position=0):
    user_2_persona = row["user 2 personas"]
    conversation = row["Best Generated Conversation"].replace("\n\n", "\n")
    turns = conversation.split("\n")
    all_generated_responses = []
    all_expected_responses = []

    for i in tqdm(range(int(len(turns) / 2)), desc="Processing turns", leave=False, position=1):
        prompt = total_prompt(df_train, user_2_persona, conversation, i, 1)
        generated_response = chat_with_model(prompt)
        all_generated_responses.append(generated_response)
        perplexity_scores_generated.append(perplexity(generated_response))
        similarity_cosine_scores_generated.append(similarity_cosine(user_2_persona, generated_response))

        expected_response = turns[i * 2 + 1].replace("User 2: ", "")
        all_expected_responses.append(expected_response)
        perplexity_scores_expected.append(perplexity(expected_response))
        similarity_cosine_scores_expected.append(similarity_cosine(user_2_persona, expected_response))

        similarity_cosine_scores.append(similarity_cosine(expected_response, generated_response))
        similarity_bert_scores.append(similarity_bert(expected_response, generated_response))

    distinct_1_scores_generated.append(distinct_n(all_generated_responses, 1))
    distinct_2_scores_generated.append(distinct_n(all_generated_responses, 2))
    distinct_1_scores_expected.append(distinct_n(all_expected_responses, 1))
    distinct_2_scores_expected.append(distinct_n(all_expected_responses, 2))

Processing rows: 100%|██████████| 10/10 [05:11<00:00, 31.13s/it]


In [ ]:
# pretrained model metrics info

print("Perplexity (lower better) generated", np.mean(perplexity_scores_generated))
print("Perplexity (lower better) expected", np.mean(perplexity_scores_expected))
print("Distinct-1 generated", np.mean(distinct_1_scores_generated))
print("Distinct-1 expected", np.mean(distinct_1_scores_expected))
print("Distinct-2 generated", np.mean(distinct_2_scores_generated))
print("Distinct-2 expected", np.mean(distinct_2_scores_expected))
print("Similarity Cosine generated", np.mean(similarity_cosine_scores_generated))
print("Similarity Cosine expected", np.mean(similarity_cosine_scores_expected))
print("Similarity Cosine", np.mean(similarity_cosine_scores))
print("Similarity Bert", np.mean(similarity_bert_scores))

Perplexity (lower better) generated 54.8181782523791
Perplexity (lower better) expected 97.72821196715037
Distinct-1 generated 0.6486132421820943
Distinct-1 expected 0.6877102684104155
Distinct-2 generated 0.9216775619832704
Distinct-2 expected 0.9248480589351887
Similarity Cosine generated 0.23713224
Similarity Cosine expected 0.213639
Similarity Cosine 0.48601323
Similarity Bert 0.9023365880052249


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = "meta-llama/Llama-3.2-3B-Instruct"
model_id = "/content/Llama-3.2-3B-Instruct"
# model_id = "/content/llama3_lora_sft"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]